# Dati mancanti {#sec-sem-missing}

In [1]:
source(here::here("code", "_common.R"))

suppressPackageStartupMessages({
    library("rio")
    library("tidyr")
})

Raramente un ricercatore si trova nella situazione fortunata nella quale un'analisi statistica (di tipo CFA/SEM o altro) può essere condotta utilizzando un set di dati in cui tutte le variabili sono state osservate su tutte le unità statistiche: nella pratica ricerca i dati mancanti sono la norma piuttosto che l'eccezione.

## Tipologie di dati mancanti

Ci sono molti motivi che possono stare alla base dei dati mancanti. Ad esempio, i dati possono mancare per disegno dello studio ("mancanza pianificata"), come ad esempio nei progetti di ricerca in cui i partecipanti al campione vengono selezionati casualmente per completare sottoinsiemi diversi della batteria di valutazione (una scelta di questo tipo viene motivata, ad esempio, a causa di considerazioni pratiche come i vincoli di tempo). In tali condizioni, si presume che i dati mancanti si distribuiscano in un modo completamente casuale rispetto a tutte le altre variabili nello studio. 

In generale, i meccanismi che determinano la presenza di dati mancanti possono essere classificati in tre categorie:

1. *valori mancanti completamente casuali* (*Missing Completely At Random*, MCAR). La probabilità di dati mancanti su una variabile non è collegata né al valore mancante sulla variabile, né al valore di ogni altra variabile presente nella matrice dati che si sta analizzando;
2. *valori mancanti casuali* (*Missing At Random*, MAR). I valori mancanti sono indipendenti dal valore che viene a mancare, ma dipendono da altre variabili, cioè i dati sulla variabile sono mancanti per categorie di partecipanti che potrebbero essere identificati dai valori assunti dalle altre variabili presenti nello studio;
3. *valori mancanti non ignorabili* (*Missing Not At Random*, MNAR). La mancanza di un dato può dipendere sia dal valore del dato stesso che dalle altre variabili. Per esempio, se si studia la salute mentale e le persone depresse riferiscono meno volentieri informazioni riguardanti il loro stato di salute, allora i dati non sono mancanti per caso.

## La gestione dei dati mancanti

Il passo successivo dopo la definizione dei meccanismi è quello della gestione dei dati mancanti. Sostanzialmente le scelte possibili sono due: l'eliminazione dei casi o la sostituzione dei dati mancanti. Un metodo semplice, indicato solo nel caso in cui l'ammontare dei dati mancanti è limitato e questi sono mancanti completamente a caso (MCAR), è quello di rimuovere i casi con dati mancanti (*case deletion*). 

Ci sono due metodi per eliminare le osservazioni con valori mancanti: *listwise deletion* e *pairwise deletion*. Nel primo caso si elimina dal campione ogni osservazione che contiene dati mancanti. Le analisi avverranno quindi solo sui casi che hanno valori validi su tutte le variabili in esame. In questo modo si ottiene una maggiore semplicità di trattazione nell'analisi statistica, tuttavia non si utilizza tutta l'informazione osservata (si riduce la numerosità campionaria e, quindi, l'informazione). Il secondo metodo è la *pairwise deletio*n, che utilizza tutti i casi che hanno i dati validi su due variabili volta per volta. In questo modo si riesce a massimizzare la numerosità del campione da utilizzare, ma si tratta comunque di un metodo che presenta dei problemi, per esempio il fatto che con questo approccio i parametri del modello saranno basati su differenti insiemi di dati, con differenti numerosità campionarie e differenti errori standard.

Quando i dati non sono MNAR è opportuno sostituirli con appropriate funzioni dei dati effettivamente osservati. Questa procedura è chiamata imputazione (*imputation*). Di seguito sono indicati alcuni metodi.

1. *Mean Imputation*. Il dato mancante viene sostituito con la media della variabile. Questo metodo, utilizzato troppo spesso per la sua semplicità, riducendo la variabilità dei dati, ha effetti importanti su molte analisi dei dati e, in generale, dovrebbe essere evitato.
2. *Regression Imputation*. Si tratta di un approccio basato sulle informazioni disponibili sulle altre variabili. Si stima una equazione di regressione lineare per ogni variabile utilizzando le altre variabili come predittori. Questo metodo offre il vantaggio di poter utilizzare i rapporti esistenti tra le variabili per effettuare le valutazioni dei dati mancanti; tuttavia esso è usato raramente, in quanto amplifica le correlazioni tra le variabili; quindi, se le analisi si basano su regressioni o modelli SEM, questo metodo è sconsigliato.
3. *Multiple Imputation*. La tecnica di multiple imputation, applicabile in caso di MAR, prevede che un dato mancante su una variabile sia sostituito, sulla base dei dati esistenti sulle altre variabili, con un valore che però comprende anche una componente di errore ricavata dalla distribuzione dei residui della variabile. 
4. *Expectation-Maximization*. Un altro approccio moderno del trattamento dei dati mancanti è l'applicazione dell'algoritmo Expectation Maximization (EM). La tecnica è quella di stimare i parametri sulla base dei dati osservati, e di stimare poi i dati mancanti sulla base di questi parametri (fase E). Poi i parametri vengono nuovamente stimati sulla base della nuova matrice di dati (fase M), e così via. Questo processo viene iterato fino a quando i valori stimati convergono. Tuttavia, una limitazione fondamentale dell'utilizzo dell'algoritmo EM per calcolare le matrici di input per le analisi CFA/SEM è che gli errori standard risultanti delle stime dei parametri non sono consistenti. Pertanto, gli intervalli di confidenza e i test di significatività possono risultare compromessi. 

### Metodo Direct ML

Benché i metodi precedenti vengano spesso usati,  nella pratica concreta è preferibile usare il metodo *Direct ML*, conosciuto anche come "raw ML" o "full information ML" (FIML), in quanto è generalmente considerano come il metodo migliore per gestire i dati mancanti nella maggior parte delle applicazioni CFA e SEM. Il metodo *full information ML* è esente dai problemi associati all'utilizzo dell'algoritmo EM e produce stime consistenti sotto l'ipotesi di normalità multivariata per dati mancanti MAR. 

Intuitivamente, l'approccio utilizza la relazione tra le variabili per dedurre quali siano i valori mancanti con maggiore probabilità. Ad esempio, se due variabili, $X$ e $Y$, sono correlate positivamente, allora se, per alcune osservazioni $i$, $X_i$ è il valore più alto nella variabile, è probabile che anche il valore mancante $Y_i$ sia un valore alto. FIML utilizza queste informazioni senza procedere all'imputazione dei valori mancanti, ma invece basandosi sulle stime più verosimili dei parametri della popolazione, ovvero massimizzando direttamente la verosimiglianza del modello specificato. Sotto l'assunzione di normalità multivariata, la funzione di verosimiglianza diventa

$$
L(\mu, \Sigma) = \prod_i f(y_i \mid \mu_i, \Sigma_i),
$$

dove $y_i$ sono i dati, $\mu_i$ e $\Sigma_i$ sono i parametri della popolazione se gli elementi mancanti in $y_i$ vengono rimossi. Si cercano i valori $\mu$ e $\Sigma$ che massimizzano la verosimiglianza.

In `lavaan` l'applicazione di tale metodo si ottiene specificando l'argomento `missing = "ml"`.

### Un esempio concreto

Per applicare il metodo *direct ML*, {cite:t}`brown2015confirmatory` prende in esame i dati reali di un questionario (un singolo fattore, quattro item, una covarianza di errore) caratterizzato dalla presenza di dati mancanti. Importiamo i dati in R:

In [2]:
d <- rio::import(here::here("data", "brown_table_9_1.csv"))
head(d)

,subject,s1,s2,s3,s4
,<int>,<int>,<int>,<int>,<int>
1,5760,2,0,1,NA
2,5761,3,3,3,NA
3,5763,2,4,4,NA
4,5761,2,0,0,NA
5,5769,2,1,1,NA
6,5771,4,3,3,NA


Abbiamo 650 osservazioni:

In [3]:
dim(d)

[1] 650   5

Le frequenze di dati mancanti vengono ottentute mediante la funzione `summary()`

In [4]:
summary(d)

    subject           s1              s2              s3              s4       
 Min.   :5756   Min.   :0.000   Min.   :0.000   Min.   :0.000   Min.   :0.000  
 1st Qu.:5934   1st Qu.:2.000   1st Qu.:2.000   1st Qu.:1.000   1st Qu.:2.000  
 Median :6102   Median :3.000   Median :3.000   Median :2.000   Median :3.000  
 Mean   :6104   Mean   :2.926   Mean   :2.563   Mean   :2.208   Mean   :2.404  
 3rd Qu.:6275   3rd Qu.:4.000   3rd Qu.:4.000   3rd Qu.:4.000   3rd Qu.:3.000  
 Max.   :6451   Max.   :4.000   Max.   :4.000   Max.   :4.000   Max.   :4.000  
                NA's   :25      NA's   :25      NA's   :25      NA's   :190    

Il modello viene specificato come segue {cite:p}`brown2015confirmatory`:

In [5]:
model <- '
  esteem =~ s1 + s2 + s3 + s4
  s2 ~~ s4
'

Adattiamo il modello ai dati specificanto l'utilizzo del metodo *full information ML* per la gestione dei dati mancanti:

In [6]:
fit <- cfa(model, data = d, missing = "fiml")

È possibile identificare le configurazioni di risposte agli item che contengono dati mancanti:

In [7]:
fit@Data@Mp[[1]]$npatterns

[1] 5

In [8]:
pats <- fit@Data@Mp[[1]]$pat * 1L
colnames(pats) <- fit@Data@ov.names[[1]]
print(pats)

     s1 s2 s3 s4
[1,]  1  1  1  1
[2,]  1  1  1  0
[3,]  0  1  1  1
[4,]  1  0  1  1
[5,]  1  1  0  1


Possiamo esaminare la proporzione di dati disponibili per ciascun indicatore e per ciascuna coppia di indicatori: 

In [9]:
coverage <- fit@Data@Mp[[1]]$coverage
colnames(coverage) <- rownames(coverage) <- fit@Data@ov.names[[1]]
print(coverage)

          s1        s2        s3        s4
s1 0.9615385 0.9230769 0.9230769 0.6692308
s2 0.9230769 0.9615385 0.9230769 0.6692308
s3 0.9230769 0.9230769 0.9615385 0.6692308
s4 0.6692308 0.6692308 0.6692308 0.7076923


Ad esempio, consideriamo l'item `s1`; se moltiplichiamo la copertura di questo elemento per la numerosità campionaria possiamo concludere che questa variabile contiene 25 osservazioni mancanti; e così via.

In [29]:
650 * 0.9615385

[1] 625

Procediamo poi come sempre per esaminare la soluzione ottenuta.

In [10]:
effectsize::interpret(fit)

Name,Value,Threshold,Interpretation
<chr>,<dbl>,<dbl>,<effctsz_>
GFI,0.999449432,0.95,satisfactory
AGFI,0.992292047,0.90,satisfactory
NFI,0.999192581,0.90,satisfactory
NNFI,0.998977535,0.90,satisfactory
CFI,0.999829589,0.90,satisfactory
RMSEA,0.020237880,0.05,satisfactory
SRMR,0.004853126,0.08,satisfactory
RFI,0.995155487,0.90,satisfactory
PNFI,0.166532097,0.50,poor


In [11]:
standardizedSolution(fit)

lhs,op,rhs,est.std,se,z,pvalue,ci.lower,ci.upper
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
esteem,=~,s1,0.7374475,0.01988465,37.086269,0.000000e+00,0.6984743,0.7764207
esteem,=~,s2,0.9204499,0.01340775,68.650574,0.000000e+00,0.8941712,0.9467286
esteem,=~,s3,0.8804116,0.01325286,66.431820,0.000000e+00,0.8544365,0.9063867
esteem,=~,s4,0.9045975,0.01632859,55.399610,0.000000e+00,0.8725941,0.9366010
s2,~~,s4,-0.8859917,0.21560986,-4.109236,3.969709e-05,-1.3085793,-0.4634042
s1,~~,s1,0.4561711,0.02932777,15.554237,0.000000e+00,0.3986898,0.5136525
s2,~~,s2,0.1527720,0.02468233,6.189531,6.034349e-10,0.1043956,0.2011485
s3,~~,s3,0.2248754,0.02333594,9.636441,0.000000e+00,0.1791378,0.2706130
s4,~~,s4,0.1817033,0.02954160,6.150760,7.711243e-10,0.1238028,0.2396038


## Dati mancanti in R

Concludiamo il capitolo con qualche breve accenno alla gestione dei dati mancanti in `R`.

In R, i valori mancanti vengono indicati dal codice `NA`, che significa *not available* — non disponibile. 

Se una variabile contiene valori mancanti, `R` non è in grado di applicare ad essa alcune funzioni, come ad esempio la media. Per questa ragione, la gran parte delle funzioni di `R` prevedono modi specifici per trattare i valori mancanti.

Ci sono diversi tipi di dati "mancanti" in `R`;

- `NA` - generico dato mancante;
- `NaN` - il codice `NaN` (*Not a Number*) indica i valori numerici impossibili, quali ad esempio un valore 0/0;
- `Inf` e `-Inf` - Infinity, si verifca, ad esempio, quando si divide un numero per 0.

La funzione `is.na()` ritorna un output che indica con TRUE le celle che contengono NA o NaN.

Si noti che 

- se `is.na(x)` è TRUE, allora `!is.na(x)` è FALSE;
- `all(!is.na(x))` ritorna TRUE se tutti i valori `x` sono NOT NA;
- `any(is.na(x))` risponde alla domanda: c'è qualche valore NA (almeno uno) in `x`?;
- `complete.cases(x)` ritorna TRUE se ciascun elemento di `x` è is NOT NA; ritorna FALSE se almeno un elemento di `x` è NA;

Le funzioni `R` `is.nan()` e `is.infinite()` si applicano ai tipi di dati `NaN` e `Inf`.

Per esempio, consideriamo il seguente data.frame:

In [12]:
d <- tibble(
  w = c(1, 2, NA, 3, NA), 
  x = 1:5, 
  y = 1, 
  z = x ^ 2 + y,
  q = c(3, NA, 5, 1, 4)
)
d

w,x,y,z,q
<dbl>,<int>,<dbl>,<dbl>,<dbl>
1,1,1,2,3
2,2,1,5,NA
NA,3,1,10,5
3,4,1,17,1
NA,5,1,26,4


In [13]:
is.na(d$w)
is.na(d$x)

[1] FALSE FALSE  TRUE FALSE  TRUE

[1] FALSE FALSE FALSE FALSE FALSE

Per creare un nuovo Dataframe senza valori mancanti:


In [14]:
d_clean <- d[complete.cases(d), ]
d_clean

w,x,y,z,q
<dbl>,<int>,<dbl>,<dbl>,<dbl>
1,1,1,2,3
3,4,1,17,1


Oppure, se vogliamo eliminare le righe con NA solo in una variabile:


In [15]:
d1 <- d[!is.na(d$q), ]
d1

w,x,y,z,q
<dbl>,<int>,<dbl>,<dbl>,<dbl>
1,1,1,2,3
NA,3,1,10,5
3,4,1,17,1
NA,5,1,26,4


Se vogliamo esaminare le righe con i dati mancanti in qualunque colonna:


In [16]:
d_na <- d[!complete.cases(d), ]
d_na

w,x,y,z,q
<dbl>,<int>,<dbl>,<dbl>,<dbl>
2,2,1,5,NA
NA,3,1,10,5
NA,5,1,26,4
